In [1]:
%reset -f

import os
import numpy as np
import pandas as pd
import random
import math
from math import e

import configparser

In [2]:
# lectura de datos
    
def load_data():
    """Lee los datos del directorio de trabajo.
    """
    global pathoutput
    global nodos, inv, vu, cartectraf
    global HSCD, PVDT, TNTR, TAMB, PDRE, FCMX, FCPR, HVUT, AVUT
    global P1FVAC1, P1FVAC2, P1FCAR1, P1FCAR2, P1FCAR3, P1FCAR4
    global P3FVAC11, P3FVAC12, P3FVAC21, P3FVAC22, P3FVAC31, P3FVAC32
    global P3FCAR11, P3FCAR12, P3FCAR13, P3FCAR21, P3FCAR22, P3FCAR31, P3FCAR32

    pathoutput = working_dir + 'output/'

    ## definicion del archivo txt donde estan los parametros
    parser = configparser.ConfigParser()
    parser.read(working_dir + 'input/params.txt')

    ## parametros globales
    HSCD = float(parser['CALCULOS']['horas_sobrecarga_dia'])
    PVDT = float(parser['CALCULOS']['perdida_vida_diaria_teorica'])
    TNTR = float(parser['CALCULOS']['temperatura_normal_trafo'])
    TAMB = float(parser['CALCULOS']['temperatura_ambiente'])
    PDRE = float(parser['CALCULOS']['probabilidad_deterioro_reubicacion'])
    FCMX = float(parser['CALCULOS']['factor_carga_maxima'])
    FCPR = float(parser['CALCULOS']['factor_carga_promedio'])

    ## calculos con parametros globales
    HVUT = 24 / PVDT
    AVUT = HVUT / (24*365) 

    ##
    ## Resoluciones 818 y 819
    ##

    ##    Transformadores monofasicos - Perdidas en vacio
    ##
    P1FVAC1 = float(parser['RES818819']['Par_1f_vac_1'])
    P1FVAC2 = float(parser['RES818819']['Par_1f_vac_2'])

    ##
    ##    Transformadores monofasicos - Perdidas con carga
    ##        
    P1FCAR1 = float(parser['RES818819']['Par_1f_car_1'])
    P1FCAR2 = float(parser['RES818819']['Par_1f_car_2'])
    P1FCAR3 = float(parser['RES818819']['Par_1f_car_3'])
    P1FCAR4 = float(parser['RES818819']['Par_1f_car_4'])

    ##
    ##    Transformadores trifasicos -- Perdidas en vacio
    ##
    P3FVAC11 = float(parser['RES818819']['Par_3f_vac_11'])
    P3FVAC12 = float(parser['RES818819']['Par_3f_vac_12'])
    P3FVAC21 = float(parser['RES818819']['Par_3f_vac_21'])
    P3FVAC22 = float(parser['RES818819']['Par_3f_vac_22'])
    P3FVAC31 = float(parser['RES818819']['Par_3f_vac_31'])
    P3FVAC32 = float(parser['RES818819']['Par_3f_vac_32'])

    ##
    ##    Transformadores trifasicos - Perdidas con carga
    ##        
    P3FCAR11 = float(parser['RES818819']['Par_3f_car_11'])
    P3FCAR12 = float(parser['RES818819']['Par_3f_car_12'])
    P3FCAR13 = float(parser['RES818819']['Par_3f_car_13'])
    P3FCAR21 = float(parser['RES818819']['Par_3f_car_21'])
    P3FCAR22 = float(parser['RES818819']['Par_3f_car_22'])
    P3FCAR31 = float(parser['RES818819']['Par_3f_car_31'])
    P3FCAR32 = float(parser['RES818819']['Par_3f_car_32'])

    ##
    ## tablas de datos
    ##
    nodos = pd.read_csv(working_dir + "input/nodos.csv", sep=',', decimal='.')
    inv = pd.read_csv(working_dir + "input/inventario_transformadores.csv", sep=',', decimal='.')
    cartectraf = pd.read_csv(working_dir + "input/carac_tecn_transf.csv", sep=',', decimal='.')
    vu = pd.read_csv(working_dir + "input/vida_util.csv", sep=',', decimal='.')

    ##
    ## nombres de las columnas de las tablas de datos
    ##
    nodos.columns = ['id_n', 'id_n_Internexa','lat','lon','tension','cpro_n','cmax_n','cremcreg','dmda_n','cens','cred','tusu','pkwh_n']
    inv.columns = ['id_t', 'id_t_Internexa','fab','fase_t','tais','capa_t','vprim','vsecu','ffab','anus','viut_t','id_n_Internexa','tacr_t','creu_t','finst']
    vu.columns = ['tgrc', 'fase_t','lipo','lspo','cpre','dura','cpor','tmpc','tmac']
    cartectraf.columns = ['fase_t', 'capa_t','cnue_t']

    ## 
    ## adecuacion de las tablas para facilidad en calculos
    ##
    inv = inv.merge(nodos[['id_n','id_n_Internexa']], on = 'id_n_Internexa',how = 'left')

    ## calcular carga maxima y carga promedio de los nodos
    nodos['cpro_n'] = nodos['dmda_n'] / 30 * FCPR
    nodos['cmax_n'] = nodos['dmda_n'] / 30 * FCMX

    ## indicar grupo del trafo para calculo de las perdidas de transformacion
    inv['grpt_t'] = 1
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 150), 'grpt_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 800), 'grpt_t'] = 3

    ## indicar grupo del trafo para calculo de las perdidas de vida util
    inv['grpv_t'] = 1
    inv.loc[(inv.fase_t == 1) & (inv.capa_t > 50), 'grpv_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 150), 'grpv_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 500), 'grpv_t'] = 3

    ## calcular vida util restante del trafo en meses
    inv['viut_t'] = AVUT
    inv['viur_t'] = (inv.viut_t - inv.anus) * 12
    inv.loc[inv.viur_t < 0, 'viur_t'] = 1

    ## indicar grupo de vida util
    vu['grpv_t'] = 1
    vu.loc[(vu.fase_t == 1) & (vu.lipo >= 50), 'grpv_t'] = 2
    vu.loc[(vu.fase_t == 3) & (vu.lipo >= 150),'grpv_t'] = 2
    vu.loc[(vu.fase_t == 3) & (vu.lipo >= 500), 'grpv_t'] = 3

    ## armar keys para busquedas
    cartectraf['faca'] = cartectraf.fase_t.map(str) + "-" + cartectraf.capa_t.map(str)
    vu['tfcg'] = vu.tgrc.map(str) + "-" + vu.fase_t.map(str) + "-" + vu.cpre.map(str) + "-" + vu['grpv_t'].map(str)


In [3]:
def bodega():
    """Calcula la bodega.
    """
    global bodini, bodfin
    
    bodini = solini[solini.id_n == 999999]
    bodfin = solfin[solfin.id_n == 999999]

In [4]:
def CostoOperSol(sol):
    sol = sol.merge(nodos[['id_n','cmax_n','pkwh_n']], on='id_n', how='left')
    sol = sol.merge(inv[['id_t','capa_t','grpt_t','fase_t','viur_t']], on='id_t', how='left')
    
    sol['cpfe_nt']=0; sol['cpcu_nt']=0; sol['futi_nt']=0; sol['pfeW']=0; sol['pcuW']=0; sol['ptrW']=0; sol['pnvac']=0; sol['pncar']=0
    sol['cperm']=0; sol['cdete']=0; sol['resta']=0

    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 1)), 'pnvac'] = P1FVAC1 * sol['capa_t'] ** P1FVAC2    
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 1)), 'pncar'] = P1FCAR1 * sol['capa_t'] ** 3 + P1FCAR2 * sol['capa_t'] ** 2 + P1FCAR3 * sol['capa_t'] + P1FCAR4
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 1)), 'pnvac'] = P3FVAC11 * sol['capa_t'] ** P3FVAC12
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 1)), 'pncar'] = P3FCAR11 * sol['capa_t'] ** 2 + P3FCAR12 * sol['capa_t'] + P3FCAR13
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 2)), 'pnvac'] = P3FVAC21 * sol['capa_t'] ** P3FVAC22
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 2)), 'pncar'] = P3FCAR21 * sol['capa_t'] + P3FCAR22
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 3)), 'pnvac'] = P3FVAC31 * sol['capa_t'] ** P3FVAC32
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 3)), 'pncar'] = P3FCAR31 * sol['capa_t'] + P3FCAR32
    
    # calculo de perdidas en hoterro y cobre, en unidades W
    sol.loc[(sol.id_n != 999999),'futi_nt'] = sol['cmax_n']/sol['capa_t']
    sol.loc[(sol.id_n != 999999),'pfeW'] = sol['pnvac']
    sol.loc[(sol.id_n != 999999),'pcuW'] = sol['pncar'] * sol['futi_nt'] ** 2
    sol.loc[(sol.id_n != 999999),'ptrW'] = sol['pfeW'] + sol['pcuW']
    
    # monetizacion de las perdidas
    sol.loc[(sol.id_n != 999999),'cpfe_nt'] = sol['pfeW'] / 1000 * sol['pkwh_n'] * 24 * 30 * sol['viur_t'] / sol['viur_t']
    sol.loc[(sol.id_n != 999999),'cpcu_nt'] = sol['pcuW'] / 1000 * sol['pkwh_n'] * 24 * 30 * sol['viur_t'] / sol['viur_t']
    sol['cpt_nt'] = sol['cpfe_nt'] + sol['cpcu_nt']

    return sol


In [5]:
def teta(sol):
    sol['key'] = str(int(TAMB)) + '-' + sol.fase_t.map(str) + '-' + sol.cpre_nt.map(str) + '-' + sol['grpv_t'].map(str)
    sol['theta'] = 0

    idkey = sol.columns.get_loc('key')
    nodo = sol.columns.get_loc('id_n')
    idfu = sol.columns.get_loc('futi_nt')
    
    for i in range(sol.shape[0]):
        if sol.iloc[i,nodo] != 999999:
            key = str(sol.iloc[i,idkey])
            tabvu = vu.loc[(vu.tfcg == key) & (vu.dura <= HSCD)]
            ncargas = tabvu.shape[0]
            carga=sol.iloc[i,idfu] * 100
            theta=0
            if carga < tabvu['cpor'].min(): theta = TNTR
            if carga >= tabvu['cpor'].max(): theta = tabvu['tmpc'].max()
            if theta == 0:
                tabvu = tabvu.sort_values(['cpor'],ascending=[False])
                for index, row in tabvu.iterrows():
                    if carga <= row['cpor']:
                        theta = row['tmpc']
                        break
            sol.at[i,'theta'] = theta
    return sol['theta']
         

In [6]:
def CosVidUt(sol):
    sol = sol.merge(nodos[['id_n','cpro_n']], on='id_n', how='left')
    sol = sol.merge(inv[['id_t','grpv_t']], on='id_t', how='left')
    sol['faca'] = sol.fase_t.map(str) + "-" + sol.capa_t.map(str)
    sol = sol.merge(cartectraf[['faca','cnue_t']], on='faca', how='left')
    sol['cvu_nt']=0; sol['viur_r']=0
    
    # calculo de la carga precedente
    sol['cpreini_nt']=0
    sol['cpre_nt']=0.9
    sol.loc[(sol.id_n != 999999),'cpre_nt'] = sol['cpro_n']/sol['capa_t']

    # aproximar la carga precedente a los valores de la norma GTC50
    sol.loc[(sol.cpreini_nt < (0.75 + 0.9) / 2),'cpre_nt'] = 0.75
    sol.loc[(sol.cpreini_nt < (0.5 + 0.75) / 2),'cpre_nt'] = 0.5

    # calcular el porcentaje diario de perdida de vida util real en porcentaje
    sol['theta'] = teta(sol).copy
    sol['theta'] = 110
    sol['fevej'] = (HSCD / 24) * (e**(15000/383 - 15000/(sol['theta'] + 273))-1)
    sol['pvdr'] = PVDT * (1 + sol['fevej'])    

    # valorar perdida de vida util restante en pesos, durante lo que queda de vida util del trafo en el nodo 
    sol.loc[(sol.id_n != 999999),'cvu_nt'] = sol['viur_t'] * 30 * sol['cnue_t'] * sol['pvdr'] / sol['viur_t']
    sol.loc[(sol.id_n != 999999),'viur_r'] = 12 / (sol['pvdr'] * 365)

    return sol

In [7]:
def CosPermutDete(sol):
    sol = sol.merge(inv[['id_t','creu_t']], on='id_t', how='left')    
    sol['resta'] = sol['id_t'] - solini['id_t']
    sol.loc[(sol.resta != 0), 'cdete'] = sol['cnue_t'] * PDRE / sol['viur_t']
    sol.loc[(sol.resta != 0), 'cperm'] = sol['creu_t'] / sol['viur_t']
    sol['csol'] = sol['cpt_nt'] + sol['cvu_nt'] + sol['cperm'] + sol['cdete']   
    return sol

In [8]:
def ndmascostoso(sol):
    # seleccionar los nodos potenciales
    solcopia = sol.copy()
    solcopia = solcopia[solcopia['eval'] == 0]
    #solcopia = solcopia.loc[(solcopia['eval'] == 0) & (solcopia['coper'] > 0)]
    solcopia = solcopia.sort_values(['csol'],ascending=[False])
    #solcopia = solcopia.sort_values(['id_n'],ascending=[True])
    #solcopia = solcopia.sort_values(['cmax_n'],ascending=[False])

    id_n = int(solcopia.iloc[0,0])
    id_t = int(solcopia.iloc[0,1])
    sol.at[sol.id_n == id_n,'eval'] = 1
    return (id_n, id_t, sol)

In [101]:
def tfmascostoso(id_n,sol):
    
    global avance, control, iteracion, permutacion
    
    pot = sol.loc[(sol['eval'] == 0)]
    tfpot = pot.shape[0]
    ctsol = sol['csol'].sum()
    id_t = int(sol.loc[(sol.id_n == id_n),'id_t'])
    
    for i in range(tfpot):
        solcopia=sol.copy()
        
        # seleccionar el primero de los trafos para hacer intercambio
        id_n2 = int(pot.iloc[i,0])
        id_t2 = int(pot.iloc[i,1])
        # hacer intercambio
        solcopia.at[(solcopia.id_n == id_n) & (solcopia.id_t == id_t),'id_t'] = id_t2
        solcopia.at[(solcopia.id_n == id_n2) & (solcopia.id_t == id_t2),'id_t'] = id_t
        
        #calcular costos
        solop = CostoOperSol(solcopia[['id_n','id_t','eval']].copy())
        solvu = CosVidUt(solop)
        solpd = CosPermutDete(solvu)
        ctsolcopia = solpd['csol'].sum()
        iteracion += 1
        if ctsolcopia < ctsol:
            permutacion += 1
            sol = solpd.copy()
            ctsol = ctsolcopia
            avance.loc[-1] = [giro,iteracion, permutacion, ctsol]
            avance.index = avance.index + 1
        control.loc[-1] = [giro,iteracion, ctsol]
        control.index = control.index + 1
    return sol 

In [102]:
def run():
   # armar la solucion inicial
    global solini, avance, control, sol, giro, iteracion, permutacion
    
    iteracion = 0; permutacion= 0
    solini=pd.DataFrame(columns=['id_n','id_t'])
    control = pd.DataFrame(columns=['giro','iter','csol'])
    avance = pd.DataFrame(columns=['giro','iter','perm','csol'])
    
    solini['id_n'] = inv['id_n']
    solini['id_t'] = inv['id_t'] 
    # calcular los costos de la solucion inicial
    sol = CostoOperSol(solini)
    sol = CosVidUt(sol)
    sol = CosPermutDete(sol)      
    sol['eval']=0
    sol.to_csv(pathoutput + 'solini.csv')
    ctsol = sol['csol'].sum()

    
    avance.loc[-1] = [0,iteracion, permutacion, ctsol]
    avance.index = avance.index + 1
    control.loc[-1] = [0,iteracion, ctsol]
    control.index = control.index + 1

    # buscar el nodo mas costoso

    ndpend = 1
    veces = 3
    giro = 0
    while veces > 0:
        giro += 1
        ndpend = 1
        while (ndpend > 0):
            id_n, id_t,sol = ndmascostoso(sol)
            sol = tfmascostoso(id_n,sol)
            ndpend = sol.loc[(sol['eval'] == 0) & ((sol['id_n'] != 999999))].shape[0]
        #reiniciar la evaluacion
        sol['eval'] = 0
        veces = veces - 1
    sol.to_csv(pathoutput + 'solfin.csv')
    avance.to_csv(pathoutput + 'avance.csv')
    control.to_csv(pathoutput + 'control.csv')
    

In [106]:
working_dir="../Tests/Test3/"
load_data()
run()

KeyboardInterrupt: 

In [90]:
sol['csol'].sum()

4198462.472664888